In [1]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [2]:
class BookInfo(BaseModel):
    book_name: str = Field(description="書籍的名字", example="百年孤獨")
    author_name: str = Field(description="書籍的作者", example="加西亞·馬爾克斯")
    genres: List[str] = Field(description="書籍的體裁", example=["小說", "文學"])

In [3]:
output_parser = PydanticOutputParser(pydantic_object=BookInfo)

In [4]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"book_name": {"description": "書籍的名字", "example": "百年孤獨", "title": "Book Name", "type": "string"}, "author_name": {"description": "書籍的作者", "example": "加西亞·馬爾克斯", "title": "Author Name", "type": "string"}, "genres": {"description": "書籍的體裁", "example": ["小說", "文學"], "items": {"type": "string"}, "title": "Genres", "type": "array"}}, "required": ["book_name", "author_name", "genres"]}
```


In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "{parser_instructions} 你輸出的結果請使用中文。"),
    ("human", "請你幫我從書籍概述中，提取書名、作者，以及書籍的體裁。書籍概述會被三個#符號包圍。\n###{book_introduction}###")
])

In [11]:
# book_introduction = """《明朝那些事兒》，作者是當年明月。2006年3月在天涯社區首次發表，2009年3月21日連載完畢，邊寫作邊集結成書出版發行，一共7本。
# 《明朝那些事兒》主要講述的是從1344年到1644年這三百年間關於明朝的一些故事。以史料為基礎，以年代和具體人物為主線，並加入了小說的筆法，語言幽默風趣。對明朝十六帝和其他王公權貴和小人物的命運進行全景展示，尤其對官場政治、戰爭、帝王心術著墨最多，並加入對當時政治經濟制度、人倫道德的演義。
# 它以一種網絡語言向讀者娓娓道出三百多年關於明朝的歷史故事、人物。其中原本在歷史中陌生、模糊的歷史人物在書中一個個變得鮮活起來。《明朝那些事兒》為讀者解讀歷史中的另一面，讓歷史變成一部活生生的生活故事。

# """
book_introduction = """《萬曆十五年》是历史学者黄仁宇的代表作，英文名为 1587, a Year of No Significance: The Ming Dynasty in Decline，首次出版于1976年。这部作品主要聚焦于明朝万历皇帝统治下的1587年，通过对这一年发生的事件的分析，探讨了明朝衰落的根本原因。
作品内容
历史背景与主题
书中以万历十五年的若干关键人物为主线，包括万历皇帝、首辅申时行、张居正、海瑞、戚继光和李贽等。黄仁宇通过这些人物的生平和所作所为，揭示了当时明朝政治、经济和社会的复杂性，以及这些因素如何影响国家的未来。尽管1587年在历史上看似无关紧要，但黄仁宇认为，这一年中的细微事件反映了整个帝国走向衰败的深层次问题。

"""
# book_introduction = """
# 《如果这是宋史》是由作者高天流云创作的一部历史小说，旨在以轻松幽默的方式讲述北宋和南宋的历史。这部作品受到《明朝那些事儿》的启发，试图通过生动的叙述风格和丰富的人物描写，让读者更好地理解宋朝的历史背景。
# 作品内容
# 叙述风格
# 《如果这是宋史》采用了小说化的叙述方式，结合历史事件与虚构情节，试图让读者在轻松的氛围中了解宋朝的政治、经济、文化等方面。书中涵盖了多个重要人物和事件，描绘了当时社会的风貌。
# """
final_prompt = prompt.invoke({"book_introduction": book_introduction, 
                              "parser_instructions": output_parser.get_format_instructions()})

In [12]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke(final_prompt)
print(response.content)

{
  "book_name": "萬曆十五年",
  "author_name": "黃仁宇",
  "genres": ["歷史", "非虛構"]
}


In [13]:
result = output_parser.invoke(response)
result

BookInfo(book_name='萬曆十五年', author_name='黃仁宇', genres=['歷史', '非虛構'])

In [14]:
result.book_name

'萬曆十五年'

In [15]:
result.genres

['歷史', '非虛構']